# Deep Q-network Implementation
Following example from Week 4 in Practical Reinforcment Learning (Coursera)

In [3]:
import gym
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext line_profiler

In [4]:
import importlib
import envs
from envs.Simple2dEnv import Simple2dEnv
importlib.reload(envs.Simple2dEnv)
from envs.Simple2dEnv import Simple2dEnv

In [16]:
# Define the parameters
dim = 2    # number of assets
trading_days_per_year = 1
risk_free_rate = 0.01
utility = lambda x : -np.exp( -2 * x )

env = Simple2dEnv(
                    dim=dim, 
                    utility=utility,  
                    risk_free_rate=risk_free_rate, 
                    trading_days_per_year=trading_days_per_year
                )

observation_shape = env.observation_space.shape
action_dim = env.action_space.shape[0]
state_dim = env.observation_space.shape

obs = env.reset()

In [15]:
import tensorflow as tf
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [ ]:
from keras.layers import Conv2D, Dense, Flatten
from keras.models import Sequential

class DQNAgent:
    def __init__(self, name, state_dim, action_dim, epsilon=0, reuse=False):
        """A simple DQN agent"""
        
        state_shape = (state_dim + action_dim,)
        
        with tf.variable_scope(name, reuse=reuse):

            self.model = Sequential([
                Dense( 16, activation='relu', input_shape=state_shape ),
                Dense( 16, activation='relu' ),
                Dense(  1, activation='sigmoid' )
            ])                 
            
            # prepare a graph for agent step
            self.state_t = tf.placeholder('float32', [None,] + list(state_shape))
            self.qvalues_t = self.get_symbolic_qvalues( self.state_t )
            
        self.weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=name)
        self.epsilon = epsilon

    def get_symbolic_qvalues(self, state_t):
        """takes agent's observation, returns qvalues. Both are tf Tensors"""
        
        #< apply your network layers here >
        #qvalues = < symbolic tensor for q-values >
        qvalues = self.model(state_t)
        
        return qvalues
    
    def get_qvalues(self, state_t):
        """Same as symbolic step except it operates on numpy arrays"""
        
        sess = tf.get_default_session()
        return sess.run(self.qvalues_t, {self.state_t: state_t})
    
    def sample_actions(self, qvalues):
        """pick actions given qvalues. Uses epsilon-greedy exploration strategy. """
        
        epsilon = self.epsilon
        batch_size, n_actions = qvalues.shape
        random_actions = np.random.uniform(n_actions, size=batch_size)
        best_actions = qvalues.argmax(axis=-1)
        should_explore = np.random.choice([0, 1], batch_size, p = [1-epsilon, epsilon])
        return np.where(should_explore, random_actions, best_actions)